In [0]:
from pyspark.sql.functions import *

# loading the csv files as Spark Dataframes
df = (
    spark.read.format("csv")
    .options(header=True, inferSchema=True)
    .load("dbfs:/FileStore/gt_data/green_tripdata_2019-01.csv")
)
df_2 = (
    spark.read.format("csv")
    .options(header=True, inferSchema=True)
    .load("dbfs:/FileStore/gt_data/taxi__zone_lookup__1_.csv")
)

# Converting two string columns into timestamps
df = df.withColumn(
    "lpep_pickup_datetime",
    to_timestamp(col("lpep_pickup_datetime"), "yyyy-MM-dd HH:mm:ss"),
).withColumn(
    "lpep_dropoff_datetime",
    to_timestamp(col("lpep_dropoff_datetime"), "yyyy-MM-dd HH:mm:ss"),
)


# Creating DeltaTables
df.write.saveAsTable("green_tbl")
df_2.write.saveAsTable("lkp_tbl")

In [0]:
%sql
SELECT
  COUNT(*) AS count
FROM
  green_tbl
WHERE
  LEFT(lpep_dropoff_datetime, 10) = '2019-01-15'
  AND LEFT(lpep_pickup_datetime, 10) = '2019-01-15'

count
20530


In [0]:
%sql
SELECT
  LEFT(lpep_pickup_datetime, 10) AS date,
  trip_distance
FROM
  green_tbl
WHERE
  trip_distance = (
    SELECT
      MAX(trip_distance)
    FROM
      green_tbl
  )

date,trip_distance
2019-01-15,117.99


In [0]:
%sql
SELECT
  SUM(
    CASE
      WHEN passenger_count = 2 THEN 1
      ELSE 0
    END
  ) AS 2_pas,
  SUM(
    CASE
      WHEN passenger_count = 3 THEN 1
      ELSE 0
    END
  ) AS 3_pas
FROM
  green_tbl
WHERE
  LEFT(lpep_pickup_datetime, 10) = '2019-01-01'

2_pas,3_pas
1282,254


In [0]:
%sql
SELECT
  l.Zone AS pickup_zone,
  l2.Zone AS drop_zone,
  g.tip_amount
FROM
  green_tbl g
  JOIN lkp_tbl l ON g.PULocationID = l.LocationID
  JOIN lkp_tbl l2 ON g.DOLocationID = l2.LocationId
WHERE
  l.Zone = "Astoria"
ORDER BY
  3 DESC
LIMIT
  1

pickup_zone,drop_zone,tip_amount
Astoria,Long Island City/Queens Plaza,88.0
